In [2]:
import time
import json

import vk
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sp

In [51]:
with open("author.txt") as f:
    login = f.readline().rstrip()
    passwd = f.readline().rstrip()

session = vk.AuthSession(app_id='6070120', user_login=login, user_password=passwd)

api = vk.API(session)
api_version = "5.73"

In [6]:
try:
    cache_dict = json.load(open('graph_cache.json', 'r'))
except FileNotFoundError:
    cache_dict = {}

In [104]:
def get_friends_with_sleep(user_id):
    global requests_count
    if requests_count == 3: # avoid making too many requests
        requests_count = 0
        time.sleep(1)

    try:
        friends = api.friends.get(user_id=user_id, v=api_version)
        requests_count += 1
    except vk.exceptions.VkAPIError as err:
        if err.code == 6: # too many requests per second
            time.sleep(2)
            friends = get_friends_with_sleep(user_id)
        else:
            raise err
    if isinstance(friends, list):
        return friends
    return friends['items']

In [105]:
def construct_graph(user_id, output_graph, api, waves=3):
    output_graph.add_node(user_id)
    if waves != 0:
        global cache_dict
        
        if user_id in cache_dict:
            friends = cache_dict[user_id]
        else:
            try:
                friends = get_friends_with_sleep(user_id)
                cache_dict[user_id] = friends
            except vk.exceptions.VkAPIError as err:
                if err.code == 18: # deleted or banned user
                    output_graph.remove_node(user_id)
                    friends = []
                else:
                    raise err
            
        for friend in friends:
            if friend not in output_graph.nodes():
                construct_graph(friend, output_graph, api, waves - 1)
            if friend in output_graph.nodes():
                output_graph.add_edge(user_id, friend)

In [ ]:
# start with my id of 10938483 for now
requests_count = 0
main_graph = nx.Graph()
start_id = 10938483
#graph_from_cache(start_id, main_graph)
construct_graph(start_id, main_graph, api, 5)

In [ ]:
len(main_graph.nodes())

In [114]:
with open('graph_cache.json', 'w') as f:
    json.dump(cache_dict, f)

In [9]:
def graph_from_cache(user_id, output_graph):
    output_graph.add_node(user_id)
    
    global cache_dict
    if user_id in cache_dict:
        friends = cache_dict[user_id]
        
        for friend in friends:
            if friend not in output_graph.nodes():
                graph_from_cache(friend, output_graph)
            output_graph.add_edge(user_id, friend)

In [110]:
nx.adjacency_matrix(main_graph)

<504410x504410 sparse matrix of type '<class 'numpy.int64'>'
	with 1141508 stored elements in Compressed Sparse Row format>

In [17]:
pos=nx.spring_layout(main_graph)

nx.draw(main_graph, pos, node_color='#A0CBE2', edge_color='#BB0000', width=2, edge_cmap=plt.cm.Blues, with_labels=True)

plt.savefig("graph.png", dpi=5000, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format=None,transparent=False, bbox_inches=None, pad_inches=0.1) 

MemoryError: 